In [ ]:
# The Yeast dataset for binary PPI prediction provided in Guo et al. 2008

In [1]:
from six.moves import cPickle as pickle #for performance

 
def save_dict(di_, filename_):
    with open(filename_, 'wb') as f:
        pickle.dump(di_, f)

def load_dict(filename_):
    with open(filename_, 'rb') as f:
        ret_di = pickle.load(f)
    return ret_di

In [2]:
prot2go = load_dict('yeast_data/protein2go_dicts.pkl')
prot2go['P40456'] = 'GO:0008150;GO:0005575;GO:0003674'
prot2go['P34234'] =  'GO:0005737;GO:0005634;GO:0005737;GO:0008150;GO:0003674'
prot2go['P53337'] = 'GO:0005789;GO:0005783;GO:0097020;GO:0030134;GO:0006888;GO:0005515;GO:0016021;GO:0016020'
prot2go['P42942'] = 'GO:0005525;GO:0000166;GO:0005737;GO:0008150;GO:0003674;GO:0016887'
prot2go['P38783'] = 'GO:0005739;GO:0003735;GO:0005763;GO:0005840;GO:0032543'
prot2go['P40187'] = 'GO:0005737;GO:0005979;GO:0019888;GO:0043666'
prot2go['P53191'] = 'GO:0005739;GO:0046872;GO:0003674;GO:0005773;GO:0005774;GO:0008150;GO:0016020'
prot2go['P35184'] = 'GO:0000027;GO:0005737;GO:0005829;GO:0042273;GO:0051082'
prot2go['P47025'] = 'GO:0000266;GO:0005741;GO:0005739;GO:0005515;GO:0000002;GO:0016020;GO:0016559;GO:0043130;GO:0090141'
prot2go['P32900'] = 'GO:0005934;GO:0016020;GO:0016021;GO:1902413;GO:0000131;GO:0003674;GO:0005935;GO:0071944'
prot2go['Q03661'] = 'GO:0005634;GO:0000784;GO:0003674;GO:0006348;GO:0034398;GO:0034399;GO:0060969'
prot2go['P33306'] = 'GO:0005515;GO:0000082;GO:0003674;GO:0005634;GO:0005737;GO:0010628;GO:0051726'
prot2go['Q04031'] = 'GO:0005730;GO:0006364;GO:0003674;GO:0003723;GO:0005634;GO:0005737;GO:0042254'

prot2go['P35723'] = 'GO:0005783;GO:0005789;GO:0016021;GO:0003674;GO:0005887;GO:0006886;GO:0006888;GO:0008150;GO:0015031;GO:0016020;GO:0016192;GO:0070973'

prot2go['Q12329'] ='GO:0005515;GO:0005737;GO:0051082;GO:0000302;GO:0006457;GO:0007010;GO:0009408;GO:0009651;GO:0030307;GO:0043621;GO:0051259;GO:0140311'

In [63]:
import numpy as np
max_golen = 0
for i,  (key, value) in enumerate(prot2go.items()):
    gos = value.split(';')
    gos = list(set(gos))
    count = 0
    for go in gos:
        if go.startswith('GO'):
            feature = np.load('../ncbi_allfeatures4go/'+go+'_0.npy')
            count += feature.shape[0]-2
    if max_golen < count:
        max_golen = count
    if len(gos) == 0:
        print(key)
    

In [64]:
max_golen

1253

In [56]:
# prot2go['Q04031'] = 'GO:0005730;GO:0006364;GO:0003674;GO:0003723;GO:0005634;GO:0005737;GO:0042254'

key

 
    
    
    
    

'P53316'

In [59]:
len(prot2go)

2497

In [3]:
from tqdm import tqdm
import requests
all_seqs = {}
unique_all_prots = list(prot2go.keys())
remain_prots = []
for t in tqdm(unique_all_prots):
    requestURL = "https://www.uniprot.org/uniprot/"+t+'.fasta'
    r = requests.get(requestURL, headers={ "Accept" : "application/json", 'User-Agent': 'User-Agent:Mozilla/5.0'})
    if r.ok:
        responseBody = r.text
        seqs = ''
        for seq in responseBody.split('\n')[1:]:
            seqs += seq.rstrip()
        if seqs == '':
            remain_prots.append(t)
        else:
            all_seqs[t] = seqs
    else:
        remain_prots.append(t)

100%|██████████| 2497/2497 [37:24<00:00,  1.11it/s] 


In [7]:
all_seqs 

{'P53049': 'MTITVGDAVSETELENKSQNVVLSPKASASSDISTDVDKDTSSSWDDKSLLPTGEYIVDRNKPQTYLNSDDIEKVTESDIFPQKRLFSFLHSKKIPEVPQTDDERKIYPLFHTNIISNMFFWWVLPILRVGYKRTIQPNDLFKMDPRMSIETLYDDFEKNMIYYFEKTRKKYRKRHPEATEEEVMENAKLPKHTVLRALLFTFKKQYFMSIVFAILANCTSGFNPMITKRLIEFVEEKAIFHSMHVNKGIGYAIGACLMMFVNGLTFNHFFHTSQLTGVQAKSILTKAAMKKMFNASNYARHCFPNGKVTSFVTTDLARIEFALSFQPFLAGFPAILAICIVLLIVNLGPIALVGIGIFFGGFFISLFAFKLILGFRIAANIFTDARVTMMREVLNNIKMIKYYTWEDAYEKNIQDIRTKEISKVRKMQLSRNFLIAMAMSLPSIASLVTFLAMYKVNKGGRQPGNIFASLSLFQVLSLQMFFLPIAIGTGIDMIIGLGRLQSLLEAPEDDPNQMIEMKPSPGFDPKLALKMTHCSFEWEDYELNDAIEEAKGEAKDEGKKNKKKRKDTWGKPSASTNKAKRLDNMLKDRDGPEDLEKTSFRGFKDLNFDIKKGEFIMITGPIGTGKSSLLNAMAGSMRKTDGKVEVNGDLLMCGYPWIQNASVRDNIIFGSPFNKEKYDEVVRVCSLKADLDILPAGDMTEIGERGITLSGGQKARINLARSVYKKKDIYLFDDVLSAVDSRVGKHIMDECLTGMLANKTRILATHQLSLIERASRVIVLGTDGQVDIGTVDELKARNQTLINLLQFSSQNSEKEDEEQEAVVAGELGQLKYESEVKELTELKKKATEMSQTANSGKIVADGHTSSKEERAVNSISLKIYREYIKAAVGKWGFIALPLYAILVVGTTFCSLFSSVWLSYWTENKFKNRPPSFYMGLYSFFVFAAFIFMNGQFTILCAMGIMASKWLNLRAVKRILHTPMSYIDTTPL

In [9]:
save_dict(all_seqs, 'yeast_data/protein2seq_dicts.pkl')

In [ ]:
save_dict(all_seqs, 'yeast_protein_seqs.pkl')

In [ ]:
import numpy as np
train_0 = np.load('yeast_data/train3.npy')

In [ ]:
train_0

In [2]:
with open('yeast_data/protein.actions.tsv', 'r') as f:
    ppi_pairs = f.readlines()[1:]
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import numpy as np
for rep in range(3):
    n_splits = 5
    y_all_1 =  np.zeros((int(len(ppi_pairs)/2),1))
    y_all_2 =  np.ones( ( len(ppi_pairs) - int(len(ppi_pairs)/2)   ,1))
    y_all = np.concatenate((y_all_1, y_all_2), axis=0)
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True)
    skf_v = StratifiedKFold(n_splits=5, shuffle=True)

    for i, (train, test) in enumerate(skf.split(ppi_pairs, y_all)):
         
        train_pairs_file = 'yeast_data/train'+str(rep)+'-'+str(i)
        test_pairs_file = 'yeast_data/test'+str(rep)+'-'+str(i)
        valid_pairs_file = 'yeast_data/valid'+str(rep)+'-'+str(i)
        valid_len = int(train.shape[0]*0.2)
        train_valid_idx = np.arange(train.shape[0])
        np.random.shuffle(train_valid_idx)
         
         
        with open(train_pairs_file,'w') as f:
            for ind in train_valid_idx[:train.shape[0]-valid_len]:
                f.writelines(ppi_pairs[ind])
         
        with open(test_pairs_file,'w') as f:
            for ind in test:
                f.writelines(ppi_pairs[ind])
        
        with open(valid_pairs_file,'w') as f:
            for ind in train_valid_idx[train.shape[0]-valid_len:]:
                f.writelines(ppi_pairs[ind])

In [ ]:
prot2go

In [ ]:
train_pairs_file

In [ ]:
ppi_pairs

In [ ]:
from sklearn.model_selection import KFold, ShuffleSplit
kf = KFold(n_splits=5, shuffle=True)
tries = 5
cur = 0
recalls = []
accuracy = []
total = []
total_truth = []
train_test = []
for train, test in kf.split(class_labels):
    if np.sum(class_labels[train], 0)[0] > 0.8 * len(train) or np.sum(class_labels[train], 0)[0] < 0.2 * len(train):
        continue
    train_test.append((train, test))
    cur += 1
    if cur >= tries:
        break

print (len(train_test))